In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Initial Data Exploration

In [ ]:
#What are the variables in the Boston Housing dataset?
Boston = pd.read_csv("Boston.csv")
Boston.columns

In [ ]:
#How many observations and variables are there in this dataset?
Boston.shape

### Below is a description of each of the variables

crim : per capita crime rate by town.

zn : proportion of residential land zoned for lots over 25,000 sq.ft.

indus : proportion of non-retail business acres per town.

chas : Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).

nox : nitrogen oxides concentration (parts per 10 million).

rm : average number of rooms per dwelling.

age : proportion of owner-occupied units built prior to 1940.

dis : weighted mean of distances to five Boston employment centres.

rad : index of accessibility to radial highways.

tax : full-value property-tax rate per \$10,000.

ptratio : pupil-teacher ratio by town.

black : 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.

lstat : lower status of the population (percent).

medv : median value of owner-occupied homes in \$1000s.

### Do any of the variables have obvious outliers or strong skew?

In [ ]:
plt.hist(Boston.crim.values, bins=50)
plt.show()

In [ ]:
plt.hist(Boston.zn.values, bins=50)
plt.show()

In [ ]:
plt.hist(Boston.black.values, bins=50)
plt.show()

It looks like crim, zn, and black are all strongly skewed, while all other variables do not appear to have any obvious outliers or extreme skew (check for yourself though).

For these three variables, we will transform them using the logarithm, remove the original version from the dataframe, and add in the transformed variables with the same name.

In [ ]:
Boston["crim"] = np.log(1 + Boston.crim.values)
Boston["zn"] = np.log(1 + Boston.zn.values)
Boston["black"] = np.log(400 - Boston.black.values)

# Model Selection

Here we wish to model the median value of homes (medv) as a function of all other variables. Furthermore, we want to find the simplest model that has the fewest number of variables in it.

To compare models we will use Mallow's $C_{p}$ defined as follows:

$$C_{p} = \dfrac{1}{n} \left( RSS + 2 p \hat{\sigma}^{2} \right)$$

where $RSS$ is the residual sum of squares, $p$ is the number of fit parameters, and $\hat{\sigma}$ is an estimate of the standard deviation of the noise. Here we will use $\hat{\sigma}^{2} = \dfrac{\sum\limits_{i=1}^{n} \left( Y_{i} - \hat{Y}_{i} \right)^{2}}{n - p}$.

### Forward Stepwise Selection

In [ ]:
variables = Boston.columns.drop(["medv"])
modelvars = []
nullmdl = sm.OLS(Boston["medv"], sm.add_constant(Boston[modelvars])).fit()
rss = np.sum((Boston.medv.values - nullmdl.fittedvalues)**2)
cp = (rss + 2*(len(modelvars)+1)*rss/(len(Boston.medv.values) - (len(modelvars)+1)))/len(Boston.medv.values)
decreasing = True
while decreasing:
    cps = []
    for var in variables:
        mdl = sm.OLS(Boston["medv"], sm.add_constant(Boston[modelvars + [var]])).fit()
        rss = np.sum((Boston.medv.values - mdl.fittedvalues)**2)
        cps.append((rss + 2*(len(modelvars)+2)*rss/(len(Boston.medv.values) - (len(modelvars)+2)))/len(Boston.medv.values))
    best = np.argmin(cps)
    if cps[best] < cp:
        modelvars.append(variables[best])
        variables.drop([variables[best]])
        cp = cps[best]
    else:
        decreasing = False

In [ ]:
#Which variables were selected as the best for this linear model through forward selection
modelvars

In [ ]:
#What does the final model look like
finalmdl = sm.OLS(Boston["medv"], sm.add_constant(Boston[modelvars])).fit()
finalmdl.summary()

### Backward Stepwise Selection

In [ ]:
modelvars2 = Boston.columns.drop(["medv"])
fullmdl = sm.OLS(Boston["medv"], sm.add_constant(Boston[modelvars2])).fit()
rss = np.sum((Boston.medv.values - fullmdl.fittedvalues)**2)
cp = (rss + 2*(len(modelvars2)+1)*rss/(len(Boston.medv.values) - (len(modelvars2)+1)))/len(Boston.medv.values)
decreasing = True
while decreasing:
    cps = []
    for var in modelvars2:
        tempmodelvars2 = list(modelvars2)
        tempmodelvars2.remove(var)
        mdl = sm.OLS(Boston["medv"], sm.add_constant(Boston[tempmodelvars2])).fit()
        rss = np.sum((Boston.medv.values - mdl.fittedvalues)**2)
        cps.append((rss + 2*(len(tempmodelvars2)+1)*rss/(len(Boston.medv.values) - (len(tempmodelvars2)+1)))/len(Boston.medv.values))
    best = np.argmin(cps)
    if cps[best] < cp:
        modelvars2 = modelvars2.drop(modelvars2[best])
        cp = cps[best]
    else:
        decreasing = False

In [ ]:
#What variables were selected by now using backward selection?
modelvars2

In [ ]:
#What does the new final model look like?
finalmdl2 = sm.OLS(Boston["medv"], sm.add_constant(Boston[modelvars2])).fit()
finalmdl2.summary()